In [1]:
from helpers import fetch_data
import polars as pl
import pandas as pd
from polars import col as c
import polars as pl
from polars import col as c
import polars.selectors as cs
import plotly.express as px

In [2]:
date_id = 120
state_filter = 'XX'  # Example state filter, replace with actual state code if needed
df = fetch_data(date_id, 'XX', 'product_group').collect()

def scatter_plot(df: pl.DataFrame):
    fig = px.scatter(
        df,
        x='rx_count',
        y='total_diff_abs',
        size='total_diff_abs',
        color='percent_change',
        title='Prescription Analysis: Total Difference vs Prescription Count',
        log_x=True,
        log_y=True,
        size_max=60,  # Slightly smaller max size to reduce overlap
        color_continuous_scale='Spectral_r',  # High contrast color scale for better visibility
    )

    # Custom hovertemplate
    hovertemplate = (
        "<b>Product:</b> %{customdata[0]}<br>"
        "<b>Classification:</b> %{customdata[1]}<br>"
        "<b>Avg Unit Change:</b> %{customdata[2]:$.2f}<br>"
        "<b>Avg New NADAC:</b> %{customdata[3]:$.2f}<br>"
        "<b>Avg Old NADAC:</b> %{customdata[4]:$.2f}<br>"
        "<b>Total Diff:</b> %{customdata[5]:$,.0f}<br>"
        "<b>Avg Diff Per Rx:</b> %{customdata[6]:$.2f}<br>"
        "<b>Rx Count:</b> %{customdata[7]:,.0f}<br>"
        "<b>Units:</b> %{customdata[8]:,.0f}<br>"
        "<b>Avg Percent Change:</b> %{customdata[9]:.1%}<br>"
        "<extra></extra>"
    )    # Prepare customdata for hovertemplate
    fig.update_traces(
        customdata=df[[
            'product_group',
            'classification',
            'avg_unit_change',
            'avg_new_nadac',
            'avg_old_nadac',
            'total_diff',
            'diff_per_rx',
            'rx_count',
            'units',
            'percent_change'
        ]].to_numpy(),        hovertemplate=hovertemplate,
        marker={
            'sizemin': 5,
              # Slightly smaller max size to reduce overlap
            'line': {'width': 2, 'color': 'rgba(44, 62, 80, 0.9)'},  # Darker, thicker borders
            'opacity': 0.7  # Reduced opacity to see overlapping bubbles
        },
    )

    fig.update_layout(
        title={
            'text': '<b>Prescription Analysis</b><br><sub>Total Difference vs Prescription Count</sub>',
            'x': 0.5,
            'xanchor': 'center',
            'font': {'size': 24, 'family': 'Inter, Segoe UI, Arial, sans-serif', 'color': '#2c3e50'}
        },
        xaxis={
            'title': {
                'text': '<b>Prescription Count</b> (log scale)', 
                'font': {'size': 16, 'family': 'Inter, Segoe UI, Arial, sans-serif', 'color': '#34495e'}
            },
            'tickformat': '~s',
            'gridcolor': 'rgba(189, 195, 199, 0.3)',
            'gridwidth': 1,
            'showline': True,
            'linecolor': 'rgba(149, 165, 166, 0.5)',
            'linewidth': 1,
            'tickfont': {'size': 12, 'color': '#7f8c8d'}
        },
        yaxis={
            'title': {
                'text': '<b>Total Difference</b> ($, log scale)', 
                'font': {'size': 16, 'family': 'Inter, Segoe UI, Arial, sans-serif', 'color': '#34495e'}
            },
            'tickformat': '$~s',
            'gridcolor': 'rgba(189, 195, 199, 0.3)',
            'gridwidth': 1,
            'showline': True,
            'linecolor': 'rgba(149, 165, 166, 0.5)',
            'linewidth': 1,
            'tickfont': {'size': 12, 'color': '#7f8c8d'}
        },        coloraxis={
            'cmin': -1,
            'cmax': 1,
            'colorscale': 'Spectral_r',  # High contrast color scale
            'colorbar': {
                'title': {
                    'text': '<b>Percent Change</b>', 
                    'font': {'size': 14, 'family': 'Inter, Segoe UI, Arial, sans-serif', 'color': '#2c3e50'}
                },
                'tickformat': '.0%',
                'orientation': 'h',
                'x': 0.5,
                'y': -0.25,
                'len': 0.8,
                'thickness': 20,
                'tickfont': {'size': 11, 'color': '#7f8c8d'},
                'bordercolor': 'rgba(149, 165, 166, 0.3)',
                'borderwidth': 1
            }
        },
        plot_bgcolor='rgba(255, 255, 255, 0.95)',  # Nearly white background for better contrast
        paper_bgcolor='white',
        font={'family': 'Inter, Segoe UI, Arial, sans-serif', 'size': 12, 'color': '#2c3e50'},
        margin={'l': 90, 'r': 90, 't': 120, 'b': 120},        width=1400,
        height=700,
        showlegend=False,
        # Configure hover mode for better interaction with overlapping points
        hovermode='closest',
        # Add subtle shadow effect
        annotations=[
            dict(
                text="",
                showarrow=False,
                xref="paper", yref="paper",
                x=0, y=0, xanchor='left', yanchor='bottom',
                xshift=-5, yshift=-5,
                bordercolor="rgba(0,0,0,0.1)",
                borderwidth=1,
                bgcolor="rgba(0,0,0,0.02)",
                width=1410, height=710
            )
        ]
    )

    return fig
scatter_plot(df)

In [ ]:
date_id = 120
state_filter = 'XX'
product_group_filter = 'Levothyroxine'  # Example state filter, replace with actual state code if needed
df = fetch_data(date_id, 'XX', 'product', product_group_filter=product_group_filter).collect()

def bar_chart(df):
    fig = px.bar(
        df.to_pandas(),
        x="total_diff",
        y="product",
        color="percent_change",
        color_continuous_scale="Spectral_r",
        orientation="h",
        title="Product-level Total Difference (Diverging Bar Chart)",
        text="total_diff",
    )

    fig.update_traces(
        texttemplate="%{x:$,.0f}",
        textposition="outside",
        marker_line_width=2,
        marker_line_color="rgba(44, 62, 80, 0.9)",
        opacity=0.8,
        customdata=df.select([
            "product",
            "classification",
            "avg_unit_change",
            "avg_new_nadac",
            "avg_old_nadac",
            "total_diff",
            "diff_per_rx",
            "rx_count",
            "units",
            "percent_change"
        ]).to_numpy(),
        hovertemplate=(
            "<b>Product:</b> %{customdata[0]}<br>"
            "<b>Classification:</b> %{customdata[1]}<br>"
            "<b>Avg Unit Change:</b> %{customdata[2]:$.2f}<br>"
            "<b>Avg New NADAC:</b> %{customdata[3]:$.2f}<br>"
            "<b>Avg Old NADAC:</b> %{customdata[4]:$.2f}<br>"
            "<b>Total Diff:</b> %{customdata[5]:$,.0f}<br>"
            "<b>Avg Diff Per Rx:</b> %{customdata[6]:$.2f}<br>"
            "<b>Rx Count:</b> %{customdata[7]:,.0f}<br>"
            "<b>Units:</b> %{customdata[8]:,.0f}<br>"
            "<b>Avg Percent Change:</b> %{customdata[9]:.1%}<br>"
            "<extra></extra>"
        )
    )

    fig.update_layout(
        title={
            "text": "<b>Product-level Total Difference</b><br><sub>Diverging Bar Chart</sub>",
            "x": 0.5,
            "xanchor": "center",
            "font": {"size": 22, "family": "Inter, Segoe UI, Arial, sans-serif", "color": "#2c3e50"}
        },
        xaxis={
            "title": {
                "text": "<b>Total Difference</b> ($)",
                "font": {"size": 16, "family": "Inter, Segoe UI, Arial, sans-serif", "color": "#34495e"}
            },
            "tickformat": "$~s",
            "gridcolor": "rgba(189, 195, 199, 0.3)",
            "gridwidth": 1,
            "showline": True,
            "linecolor": "rgba(149, 165, 166, 0.5)",
            "linewidth": 1,
            "tickfont": {"size": 12, "color": "#7f8c8d"}
        },
        yaxis={
            "title": {
                "text": "<b>Product</b>",
                "font": {"size": 16, "family": "Inter, Segoe UI, Arial, sans-serif", "color": "#34495e"}
            },
            "tickfont": {"size": 12, "color": "#7f8c8d"},
            "categoryorder": "total ascending"
        },
        coloraxis={
            "cmin": -1,
            "cmax": 1,
            "colorscale": "Spectral_r",
            "colorbar": {
                "title": {
                    "text": "<b>Percent Change</b>",
                    "font": {"size": 14, "family": "Inter, Segoe UI, Arial, sans-serif", "color": "#2c3e50"}
                },
                "tickformat": ".0%",
                "orientation": "h",
                "x": 0.5,
                "y": -0.25,
                "len": 0.8,
                "thickness": 20,
                "tickfont": {"size": 11, "color": "#7f8c8d"},
                "bordercolor": "rgba(149, 165, 166, 0.3)",
                "borderwidth": 1
            }
        },
        plot_bgcolor="rgba(255, 255, 255, 0.95)",
        paper_bgcolor="white",
        font={"family": "Inter, Segoe UI, Arial, sans-serif", "size": 12, "color": "#2c3e50"},
        margin={"l": 120, "r": 60, "t": 100, "b": 80},
        width=1200,
        height=600,
        showlegend=False,
        hovermode="closest"
    )

    fig.show()

bar_chart()


In [4]:
pl.scan_parquet('data/date_id.parquet').collect()

date_id,effective_date,date_filter
u32,date,str
1,2013-03-01,"""2013-03"""
2,2013-04-01,"""2013-04"""
3,2013-05-01,"""2013-05"""
4,2013-06-01,"""2013-06"""
5,2013-07-01,"""2013-07"""
…,…,…
143,2025-01-01,"""2025-01"""
144,2025-02-01,"""2025-02"""
145,2025-03-01,"""2025-03"""


In [5]:
date_id = 147
state_filter = 'XX'  # Example state filter, replace with actual state code if needed
df = fetch_data(date_id, 'XX', 'product', product_group_filter='Mesalamine').collect()
df

product,units,rx_count,total_diff,new_nadac,old_nadac,avg_new_nadac,avg_old_nadac,diff_per_rx,total_diff_abs,diff_per_rx_abs,classification,avg_unit_change,percent_change
str,f64,f64,i64,f64,f64,f64,f64,f64,i64,f64,str,f64,f64
"""Mesalamine Tab Delayed Release…",1.526425e6,11378.0,-1108592,7.8296e6,8.9382e6,5.1294,5.8557,-97.43,1108592,97.43,"""Decrease""",-0.73,-0.124
"""Mesalamine Cap ER 24HR 0.375 G…",2.0088e6,14220.0,-439276,965974.97,1.4053e6,0.4809,0.6996,-30.89,439276,30.89,"""Decrease""",-0.22,-0.3126
"""Mesalamine Cap ER 500 MG""",465485.0,2887.0,-10659,1.6474e6,1.6581e6,3.5391,3.562,-3.69,10659,3.69,"""Decrease""",-0.02,-0.0064
"""Mesalamine Cap DR 400 MG""",1.969103e6,11202.0,387446,4.1972e6,3.8097e6,2.1315,1.9348,34.59,387446,34.59,"""Increase""",0.2,0.1017
"""Mesalamine Tab Delayed Release…",6.817153e6,57000.0,-4455394,8.2141e6,1.2670e7,1.2049,1.8585,-78.16,4455394,78.16,"""Decrease""",-0.65,-0.3517
